# Market Making with Order Book Imbalance

This notebook implements an imbalance-driven quoting model with inventory control.

**Outline**:
1. Imports & data generation
2. L2 book synthesis
3. Imbalance computation $I=\frac{\sum b - \sum a}{\sum b + \sum a}$
4. Spread & skew policy
5. Inventory penalty
6. Simulation loop
7. Fill simulator (depth aware)
8. PnL report
9. Sensitivity analysis
10. Plots & discussion

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(0)


## 1. Price path & microstructure noise

In [ ]:
T=2000
drift=0.0; vol=0.1
mid=100+np.cumsum(drift+vol*np.random.randn(T))
spread_base=0.04+0.02*np.abs(np.random.randn(T))


## 2. Synthetic L2 order book
We build 10 levels per side with decaying sizes.

In [ ]:
L=10
bids=np.stack([mid - (i+1)*spread_base for i in range(L)],axis=1)
asks=np.stack([mid + (i+1)*spread_base for i in range(L)],axis=1)
size_curve=lambda i: 2.0*np.exp(-0.2*i)
bid_sz=np.stack([size_curve(i)*np.ones(T) for i in range(L)],axis=1)
ask_sz=np.stack([size_curve(i)*np.ones(T) for i in range(L)],axis=1)


## 3. Imbalance $I_t$

In [ ]:
B=bid_sz.sum(1); A=ask_sz.sum(1)
imb=(B-A)/(B+A+1e-9)


## 4. Quoting policy
We use a spread `s` and skew proportional to imbalance: $q_{bid}=mid - (s/2)(1+\kappa I)$, $q_{ask}=mid + (s/2)(1-\kappa I)$.

In [ ]:
kappa=0.6
s=2*spread_base
q_bid=mid - 0.5*s*(1+kappa*imb)
q_ask=mid + 0.5*s*(1-kappa*imb)


## 5. Inventory control
Penalize inventory deviations using linear feedback: $\Delta q = \phi\, inv$.

In [ ]:
phi=0.01
inv=np.zeros(T)
q_bid_adj=q_bid.copy(); q_ask_adj=q_ask.copy()
for t in range(1,T):
    q_bid_adj[t]-=phi*inv[t-1]
    q_ask_adj[t]+=phi*inv[t-1]


## 6. Depth-aware execution simulator
We cross the book when adverse ticks occur; passive fills when our quotes improve top levels.

In [ ]:
def simulate_fills(qb, qa, bids_t, asks_t, bsz_t, asz_t, inv_prev):
    # market orders arrive stochastically
    lam=0.2
    takes_buy = np.random.rand()<lam*(1+max(0, -imb_t))*0.5
    takes_sell= np.random.rand()<lam*(1+max(0,  imb_t))*0.5
    fill_b=0.0; fill_a=0.0; cash=0.0
    # passive: if our bid is inside top-of-book, we get some fraction
    if qb>=bids_t[0]:
        qty=min(0.3*bsz_t[0], 1.0)
        fill_b+=qty; cash-=qty*qb
    if qa<=asks_t[0]:
        qty=min(0.3*asz_t[0], 1.0)
        fill_a+=qty; cash+=qty*qa
    # aggressive: react to takes
    if takes_buy:
        # we sell to buyer at our ask if <= ask best
        if qa<=asks_t[0]:
            qty=0.5
            fill_a+=qty; cash+=qty*qa
    if takes_sell:
        if qb>=bids_t[0]:
            qty=0.5
            fill_b+=qty; cash-=qty*qb
    return fill_b, fill_a, cash


## 7. Simulation loop

In [ ]:
cash=0.0
pnl=np.zeros(T)
for t in range(T):
    global imb_t
    imb_t=float(imb[t])
    fb, fa, dc = simulate_fills(q_bid_adj[t], q_ask_adj[t], bids[t], asks[t], bid_sz[t], ask_sz[t], inv[t-1] if t>0 else 0.0)
    inv[t]=(inv[t-1] if t>0 else 0.0)+fb-fa
    cash+=dc
    pnl[t]=cash+inv[t]*mid[t]


## 8. Report

In [ ]:
ret=np.diff(pnl,prepend=pnl[0])
sharpe=np.mean(ret)/np.std(ret+1e-9)*np.sqrt(252*6*60) # rough
print('Final PnL:', pnl[-1], 'Sharpe~', sharpe)


## 9. Sensitivity to $\kappa$

In [ ]:
def run_kappa(k):
    q_bid=mid - 0.5*s*(1+k*imb)
    q_ask=mid + 0.5*s*(1-k*imb)
    inv=0.0; cash=0.0
    for t in range(T):
        qb=q_bid[t]-phi*inv; qa=q_ask[t]+phi*inv
        fb,fa,dc=simulate_fills(qb,qa,bids[t],asks[t],bid_sz[t],ask_sz[t],inv)
        inv+=fb-fa; cash+=dc
    return cash
for k in [0.2,0.4,0.6,0.8,1.0]:
    print(k, run_kappa(k))


## 10. Plots

In [ ]:
plt.figure(); plt.plot(pnl); plt.title('PnL');
plt.figure(); plt.plot(inv); plt.title('Inventory');
plt.figure(); plt.plot(mid,label='mid'); plt.plot(q_bid_adj,label='q_bid_adj'); plt.plot(q_ask_adj,label='q_ask_adj'); plt.legend(); plt.title('Quotes vs Mid'); plt.show()
